In [1]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import torch.nn.functional as F
import tqdm

In [2]:
import logging
def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger

In [3]:
from torchmetrics import F1
from torchmetrics import Accuracy

In [4]:
import torch.nn as nn
import torch



class CNN(nn.Module):
    def __init__(
        self,
        n_in_channel=1,
        activation="Relu",
        conv_dropout=0,
        kernel_size=[3, 3, 3, 3, 3, 3],
        padding=[1, 1, 1, 1, 1, 1],
        stride=[1, 1, 1, 1, 1, 1],
        nb_filters=[16, 32, 64, 128, 256, 256],
        pooling=[(2, 2), (2, 2), (1, 2), (1, 2), (1, 2), (1, 2)],
        normalization="batch",
        **transformer_kwargs
    ):
        """
            Initialization of CNN network s
        
        Args:
            n_in_channel: int, number of input channel
            activation: str, activation function
            conv_dropout: float, dropout
            kernel_size: kernel size
            padding: padding
            stride: list, stride
            nb_filters: number of filters
            pooling: list of tuples, time and frequency pooling
            normalization: choose between "batch" for BatchNormalization and "layer" for LayerNormalization.
        """
        super(CNN, self).__init__()

        self.nb_filters = nb_filters
        cnn = nn.Sequential()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(264,132)
        self.fc2 = nn.Linear(132,3)
        self.fc_age = nn.Linear(6,3)
        self.fc_sex = nn.Linear(2,1)
        self.fc_hw = nn.Linear(2,1)
        self.fc_preg = nn.Linear(2,1)
        self.fc_loc = nn.Linear(5,2)
        

        def conv(i, normalization="batch", dropout=None, activ="relu"):
            nIn = n_in_channel if i == 0 else nb_filters[i - 1]
            nOut = nb_filters[i]
            cnn.add_module(
                "conv{0}".format(i),
                nn.Conv2d(nIn, nOut, kernel_size[i], stride[i], padding[i]),
            )
            if normalization == "batch":
                cnn.add_module(
                    "batchnorm{0}".format(i),
                    nn.BatchNorm2d(nOut, eps=0.001, momentum=0.99),
                )
            elif normalization == None:
                pass
                
            elif normalization == "layer":
                cnn.add_module("layernorm{0}".format(i), nn.GroupNorm(1, nOut))

            if activ.lower() == "leakyrelu":
                cnn.add_module("relu{0}".format(i), nn.LeakyReLU(0.2))
            elif activ.lower() == "relu":
                cnn.add_module("relu{0}".format(i), nn.ReLU())
            elif activ.lower() == "glu":
                cnn.add_module("glu{0}".format(i), GLU(nOut))
            elif activ.lower() == "cg":
                cnn.add_module("cg{0}".format(i), ContextGating(nOut))

            if dropout is not None:
                cnn.add_module("dropout{0}".format(i), nn.Dropout(dropout))
            elif dropout is None:
                pass

        # 128x862x64
        for i in range(len(nb_filters)):
            conv(i, normalization=normalization, dropout=conv_dropout, activ=activation)
            cnn.add_module(
                "pooling{0}".format(i), nn.MaxPool2d(pooling[i])
            )  # bs x tframe x mels

        self.cnn = cnn

    def forward(self, mel, age, sex, hw, preg, loc):
        """
        Forward step of the CNN module
        Args:
            x (Tensor): input batch of size (batch_size, n_channels, n_frames, n_freq)
        Returns:
            Tensor: batch embedded
        """
        # conv features
        mel = mel.unsqueeze(1)
        mel = self.cnn(mel)
        mel = self.avgpool(mel)
        mel = mel.view(mel.size(0), -1)
#         print(mel.shape)
        age = self.fc_age(age)
#         print(age.shape)
        sex = self.fc_sex(sex)
#         print(sex.shape)
        hw = self.fc_hw(hw)
#         print(hw.shape)
        preg = self.fc_preg(preg)
#         print(preg.shape)
        loc = self.fc_loc(loc)
#         print(loc.shape)
        x = torch.concat([mel,age,sex,hw,preg,loc],axis=1)
#         print(x.shape)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x =self.fc2(x)
        return x

In [5]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument('--output_folder',    default='./output_folder',     help='output folder')
parser.add_argument("--label_type",default="murmur", type=str, help="label type")
parser.add_argument("--lr",default=1e-3, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=64, type=int)
parser.add_argument("--minibatchsize_valid", default=1, type=int)
parser.add_argument("--train_num_workers", default=6, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_CNN', type=str)
parser.add_argument("--logdir", default='./log_hmd/', type=str)
parser.add_argument("--model_name", default='hmd_CNN_model', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=200, type=int)
parser.add_argument("--gpu", default="0", type=str)




warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [6]:
from torch.utils.data import Dataset,DataLoader, RandomSampler
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
import librosa
import glob
import math






def read_recording(wav_fn, audio_length= 20, sr= 4000):

    recording, frequency = load_wav_file(wav_fn)


    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording



class train_loader(Dataset):
    def __init__(self,data_folder, **kwargs):
        self.data_folder = data_folder
        
        self.wav_files = glob.glob(data_folder + '*.wav')
        self.num_wav_files = len(self.wav_files)
        
        self.murmur_classes = ['Present', 'Unknown', 'Absent']
        self.num_murmur_classes = len(self.murmur_classes)
        self.outcome_classes = ['Abnormal', 'Normal']
        self.num_outcome_classes = len(self.outcome_classes)
        
        self.age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
        self.recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

        
        
        examples = {}
        for filename in self.wav_files:
            current_patient_id = filename.split('/')[-1].split('_')[0]
            current_txt_file = self.data_folder + str(current_patient_id) + '.txt'
            with open(current_txt_file, 'r') as f:
                current_patient_data = f.read()
            current_loc = filename.split('/')[-1].split('.')[0].split('_')[1]
            if filename.split('/')[-1].split('.')[0]  not in examples.keys():
                current_wav_data = filename.split('/')[-1].split('.')[0]
                examples[current_wav_data] = {
                    "id" : current_patient_id,
                    "txt_fn" : current_txt_file,   #txt,
                    "wav_fn" : filename,  #wav
                    "num_location": get_num_locations(current_patient_data),
                    "frequency" : get_frequency(current_patient_data),
                    "location" : current_loc,
                    "age" : get_age(current_patient_data),
                    "sex" : get_sex(current_patient_data),
                    "height" : get_height(current_patient_data),
                    "weight" : get_weight(current_patient_data),
                    "pregnancy_status" : get_pregnancy_status(current_patient_data),
                    "murmur" : get_murmur(current_patient_data),
                    "outcome" : get_outcome(current_patient_data)

                }
        self.examples = examples
        self.examples_list = list(examples.keys())
        
        
        if len(self.examples_list) != self.num_wav_files:
            raise ValueError('length is different!')
    def __len__(self):
        return len(self.examples_list)
    
        
    def __getitem__(self, index):
        
        # patient info
        current_wav_dict = self.examples[self.examples_list[index]]
        current_patient_data = load_patient_data(current_wav_dict["txt_fn"])
        current_recording = read_recording(current_wav_dict["wav_fn"])
        
        #log mel feature
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = current_recording.astype(np.float32),
                                                         sr= 4000,
                                                         n_mels=128,
                                                         n_fft=400, 
                                                         hop_length=128, 
                                                         win_length=400))
        
        # age
        current_patient_age = current_wav_dict["age"]
        current_age_group = np.zeros(6, dtype=np.float32)
        if current_patient_age in self.age_classes:
            j = self.age_classes.index(current_patient_age)
            current_age_group[j] = 1.0
        else :
            current_age_group[5] = 1.0
            
            
        # sex
        sex = current_wav_dict["sex"]
        sex_feature = np.zeros(2, dtype=np.float32)
        if compare_strings(sex, 'Female'):
            sex_feature[0] = 1.0
        elif compare_strings(sex, 'Male'):
            sex_feature[1] = 1.0
            
        # height and weight.
        height = current_wav_dict["height"]
        weight = current_wav_dict["weight"]
        
        ## simple impute
        if math.isnan(height) :
            height = 110.846  #mean
        if math.isnan(weight) :
            weight = 23.767   #mean
            
        height_weight = np.array([height, weight], dtype=np.float32)
        
        
        # Extract pregnancy
        preg_feature = np.zeros(2, dtype=np.float32)
        is_pregnant = current_wav_dict["pregnancy_status"]
        if is_pregnant == True:
            preg_feature[0] = 1.0
        elif is_pregnant == False:
            preg_feature[1] = 1.0

        # Extract location
        location = current_wav_dict["location"]
        num_recording_locations = len(self.recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if location in self.recording_locations:
            j = self.recording_locations.index(location)
            loc_feature[j] = 1.0


        

        # label
        
        current_murmur = np.zeros(self.num_murmur_classes, dtype=np.float32)
        murmur = get_murmur(current_patient_data)
        if murmur in self.murmur_classes:
            j = self.murmur_classes.index(murmur)
            current_murmur[j] = 1


        current_outcome = np.zeros(self.num_outcome_classes, dtype=np.float32)
        outcome = get_outcome(current_patient_data)
        if outcome in self.outcome_classes:
            j = self.outcome_classes.index(outcome)
            current_outcome[j] = 1
            


        
        return log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome


In [13]:
def train_challenge_model(data_folder, model_folder, verbose):
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print ('Available devices ', torch.cuda.device_count())
        
    GPU_NUM = args.gpu # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)
    os.makedirs(args.logdir, exist_ok=True)

    open(args.logdir + 'score.txt', "a+")
    score_file = open(args.logdir + 'score.txt', "a+")
    
    
    model_folder = model_folder
    log_dir = args.logdir
    logger = get_logger(log_dir + '/' + model_folder)
    
    # Define the model
    cnn = CNN()
    cnn = cnn.cuda(device)
    
    # Define the metrics
    accuracy = Accuracy(num_classes=3, multiclass=True)
    f1 = F1(num_classes=3,average='macro', multiclass=True)
    
    # optimizer, loss
    opt = torch.optim.Adam(cnn.parameters(), lr=args.lr, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
    criterion = torch.nn.CrossEntropyLoss()
    
    
    #data loader
    trainset = train_loader(data_folder)
    validset = train_loader(args.valid_data_folder)
    point_sampler = RandomSampler(trainset)
    
    dataloader_train = DataLoader(dataset=trainset,
                        batch_size=args.minibatchsize_train,
                               sampler =point_sampler,
                               num_workers=args.train_num_workers)
    dataloader_valid = DataLoader(dataset=validset,
                        batch_size=args.minibatchsize_valid,
                               num_workers=args.dev_num_workers)
    
    # Train the model.
    if verbose >= 1:
        print('Training model...')
        
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    if args.label_type=='murmur':
        classes =  murmur_classes 
    elif args.label_type =='outcome':
        classes = outcome_classes
    
    
        
    all_file_train = len(trainset)
    dataloader_train_len = len(dataloader_train)
    output_preds_labels_train = torch.empty((all_file_train,))
    output_preds_probs_train = torch.empty((all_file_train,3))
    output_gt_labels_train = torch.empty((all_file_train,))

    for iter_ in range(args.end_iter):
        start_time = time.time()
        running_loss = 0.0
        idx = 0
        cnn.train()
        for log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome in tqdm.tqdm(dataloader_train):
            log_mel_feature = log_mel_feature.cuda()
            current_age_group = current_age_group.cuda()
            sex_feature = sex_feature.cuda()
            height_weight = height_weight.cuda()
            preg_feature = preg_feature.cuda()
            loc_feature = loc_feature.cuda()
            current_murmur = current_murmur.cuda()
            current_outcome = current_outcome.cuda()




            output_train = cnn(log_mel_feature,current_age_group,sex_feature,height_weight, preg_feature,loc_feature)
            loss = criterion(output_train, current_murmur)



            output_probs_train = F.log_softmax(output_train, dim=1)

            _, output_label_argmax = torch.max(output_probs_train, dim=1)


            output_label_argmax = output_label_argmax.data.detach().cpu()

            output_preds_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train]= output_label_argmax  #murmur 예측 레이블 (0,1,2)

            output_preds_probs_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train,:] = output_probs_train
            
            

            _, murmur_label_argmax = torch.max(current_murmur.data, dim=1)  #murmur groud truth (0,1,2)

            output_gt_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train] = murmur_label_argmax.data.detach().cpu()



            idx +=1              

            opt.zero_grad()
            loss.backward()
            opt.step()

            running_loss += loss.item()

        train_f1 = f1(preds = torch.tensor(output_preds_labels_train, dtype=torch.int32), target = torch.tensor(output_gt_labels_train, dtype=torch.int32))
        train_acc = accuracy(preds = torch.tensor(output_preds_labels_train, dtype=torch.int32), target = torch.tensor(output_gt_labels_train, dtype=torch.int32))
        

        scheduler.step()
        print("lr: ", opt.param_groups[0]['lr'])
        logger.info("Iteration:{0}, train loss = {1:.6f} ,train F1 = {2:.6f} ,train ACC = {3:.6f} ".format(iter_, 
                     running_loss/dataloader_train_len,train_f1,train_acc))

        # eval
        all_file_valid = len(validset)
        dataloader_valid_len = len(dataloader_valid)
        cnn.eval()
        
        with torch.no_grad():
            running_loss_dev = 0.0
            index = 0
            output_preds_labels_dev = torch.empty((all_file_valid,))
            output_preds_labels_onehot_dev = torch.zeros((all_file_valid,3))
            output_preds_probs_dev = torch.empty((all_file_valid,3))
            output_gt_labels_dev= torch.empty((all_file_valid,))
            output_gt_labels_onehot_dev = torch.empty((all_file_valid,3))
            for log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome in tqdm.tqdm(dataloader_valid):
                log_mel_feature_dev = log_mel_feature.cuda()
                current_age_group_dev = current_age_group.cuda()
                sex_feature_dev = sex_feature.cuda()
                height_weight_dev = height_weight.cuda()
                preg_feature_dev = preg_feature.cuda()
                loc_feature_dev = loc_feature.cuda()
                current_murmur_dev = current_murmur.cuda()
                current_outcome_dev = current_outcome.cuda()


                output_dev = cnn(log_mel_feature_dev,current_age_group_dev,sex_feature_dev,height_weight_dev,preg_feature_dev,loc_feature_dev)
                loss_dev = criterion(output_dev, current_murmur_dev)                   
                running_loss_dev += loss_dev.item()




                output_probs_dev = F.log_softmax(output_dev, dim=1)

                _, output_label_argmax_dev = torch.max(output_probs_dev, dim=1)
                
                output_preds_labels_onehot_dev[index,output_label_argmax_dev]=1


                output_label_argmax_dev = output_label_argmax_dev.data.detach().cpu()

                output_preds_labels_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid]= output_label_argmax_dev  #murmur 예측 레이블 (0,1,2)

                output_preds_probs_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid,:] = output_probs_dev

                _, murmur_label_argmax_dev = torch.max(current_murmur_dev.data, dim=1)  #murmur groud truth (0,1,2)

                output_gt_labels_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid] = murmur_label_argmax_dev
                
                output_gt_labels_onehot_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid] = current_murmur_dev

                index = index+1   
                
            
            
            outputs = output_preds_labels_onehot_dev.data.detach().cpu().numpy()
            labels = output_gt_labels_onehot_dev.data.detach().cpu().numpy()


            dev_f1 = f1(preds = torch.tensor(output_preds_labels_dev, dtype=torch.int32), target = torch.tensor(output_gt_labels_dev, dtype=torch.int32))
            dev_acc = accuracy(preds = torch.tensor(output_preds_labels_dev, dtype=torch.int32),target =  torch.tensor(output_gt_labels_dev, dtype=torch.int32))
            weighted_acc = compute_weighted_accuracy(labels, outputs, classes)

            



        logger.info("valid Epoch:%d, Train loss=%.4f, Valid loss=%.4f,Valid F1=%.4f,Valid ACC=%.4f, Valid weighted acc=%.4f" %( iter_,
                running_loss/dataloader_train_len, running_loss_dev/dataloader_valid_len, dev_f1, dev_acc,weighted_acc ))
        score_file.write("%d epoch, VALID LOSS %f, VALID F1 %2.2f%% VALID ACC %2.2f%% VALID WEIGHTED ACC%2.2f%%\n"%(iter_, 
                                running_loss_dev/dataloader_valid_len, dev_f1, dev_acc,weighted_acc ))
        score_file.flush()

        torch.save(cnn.state_dict(), os.path.join(model_folder, "{}_{}.model".format(args.model_name, iter_)))

        end_time = time.time()
        logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
        logger.info("*" * 50)


    if verbose >= 1:
        print('Done.')

In [14]:
train_challenge_model(args.train_data_folder, args.model_path, verbose=2)

  0%|                                                    | 0/40 [00:00<?, ?it/s]

Finding data files...
Available devices  1
Training model...


100%|███████████████████████████████████████████| 40/40 [00:15<00:00,  2.51it/s]

lr:  0.00091
07-28 23:51 [MainProcess, 3068] [INFO ]  Iteration:0, train loss = 0.643706 ,train F1 = 0.407393 ,train ACC = 0.767772 
07-28 23:51 [MainProcess, 3068] [INFO ]  Iteration:0, train loss = 0.643706 ,train F1 = 0.407393 ,train ACC = 0.767772 
07-28 23:51 [MainProcess, 3068] [INFO ]  Iteration:0, train loss = 0.643706 ,train F1 = 0.407393 ,train ACC = 0.767772 



100%|████████████████████████████████████████| 631/631 [00:05<00:00, 112.89it/s]

...

07-29 00:25 [MainProcess, 3068] [INFO ]  valid Epoch:87, Train loss=0.1784, Valid loss=0.5222,Valid F1=0.5878,Valid ACC=0.8510, Valid weighted acc=0.7523
07-29 00:25 [MainProcess, 3068] [INFO ]  valid Epoch:87, Train loss=0.1784, Valid loss=0.5222,Valid F1=0.5878,Valid ACC=0.8510, Valid weighted acc=0.7523
07-29 00:25 [MainProcess, 3068] [INFO ]  valid Epoch:87, Train loss=0.1784, Valid loss=0.5222,Valid F1=0.5878,Valid ACC=0.8510, Valid weighted acc=0.7523
07-29 00:25 [MainProcess, 3068] [INFO ]  Time used for each epoch training: 23.796895265579224 seconds.
07-29 00:25 [MainProcess, 3068] [INFO ]  Time used for each epoch training: 23.796895265579224 seconds.
07-29 00:25 [MainProcess, 3068] [INFO ]  Time used for each epoch training: 23.796895265579224 seconds.
07-29 00:25 [MainProcess, 3068] [INFO ]  **************************************************
07-29 00:25 [MainProcess, 3068] [INFO ]  **************************************************
07-29 00:25 [MainProcess, 3068] [INFO ]  *

100%|███████████████████████████████████████████| 40/40 [00:16<00:00,  2.36it/s]

lr:  2.2629966787881234e-07
07-29 00:25 [MainProcess, 3068] [INFO ]  Iteration:88, train loss = 0.168649 ,train F1 = 0.823125 ,train ACC = 0.939179 
07-29 00:25 [MainProcess, 3068] [INFO ]  Iteration:88, train loss = 0.168649 ,train F1 = 0.823125 ,train ACC = 0.939179 
07-29 00:25 [MainProcess, 3068] [INFO ]  Iteration:88, train loss = 0.168649 ,train F1 = 0.823125 ,train ACC = 0.939179 



 69%|███████████████████████████▍            | 433/631 [00:03<00:01, 117.90it/s]

KeyboardInterrupt: 

In [ ]:
def get_features(data, recordings):
    
    
    


    return 

In [ ]:
def load_challenge_model(model_folder, verbose):
    murmur_train_model = torch.load(model_folder + '/' + args.murmur_model_name + '_63.model')
    outcome_train_model = torch.load(model_folder + '/' + args.outcome_model_name + '_63.model')
    return [murmur_train_model,outcome_train_model]

# Run your trained model. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def run_challenge_model(model, data, recordings, verbose):
    murmur_model , outcome_model = model
    murmur_classifier.load_state_dict(murmur_model)
    outcome_classifier.load_state_dict(outcome_model)
    
    wav_files = glob.glob(data_folder + '*.wav')
    num_wav_files = len(wav_files)
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    
    
    
    
    

    
    
   
    # Get classifier probabilities.
    murmur_outputs = murmur_classifier(log_mel_feature_dev,current_age_group_dev,sex_feature_dev,height_weight_dev,preg_feature_dev,loc_feature_dev)
    murmur_probabilities = np.asarray(murmur_probabilities, dtype=np.float32)[:, 0, 1]
    outcome_probabilities = outcome_classifier.predict_proba(features)
    outcome_probabilities = np.asarray(outcome_probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

################################################################################
#
# Optional functions. You can change or remove these functions and/or add new functions.
#
################################################################################

# Save your trained model.
# def save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier):
#     d = {'imputer': imputer, 'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier, 'outcome_classes': outcome_classes, 'outcome_classifier': outcome_classifier}
#     filename = os.path.join(model_folder, 'model.sav')
#     joblib.dump(d, filename, protocol=0)

In [3]:
patient_files = find_patient_files(args.train_data_folder)

In [15]:
patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [7]:
os.makedirs(args.output_folder, exist_ok=True)

In [8]:
num_patient_files = len(patient_files)

In [12]:
patient_data = load_patient_data(patient_files[0])
recordings = load_recordings(args.train_data_folder, patient_data)

In [13]:
head, tail = os.path.split(patient_files[0])
root, extension = os.path.splitext(tail)
output_file = os.path.join(args.output_folder, root + '.csv')
patient_id = get_patient_id(patient_data)


In [14]:
output_file

'./output_folder/2530.csv'

In [44]:
a = torch.zeros((10,3))

In [45]:
a[3:8,torch.tensor(0)]=1

In [46]:
a

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
def save_challenge_outputs(filename, patient_id, classes, labels, probabilities):
    # Format Challenge outputs.
    patient_string = '#{}'.format(patient_id)
    class_string = ','.join(str(c) for c in classes)
    label_string = ','.join(str(l) for l in labels)
    probabilities_string = ','.join(str(p) for p in probabilities)
    output_string = patient_string + '\n' + class_string + '\n' + label_string + '\n' + probabilities_string + '\n'

    # Write the Challenge outputs.
    with open(filename, 'w') as f:
        f.write(output_string)


In [20]:
compute_weighted_accuracy(labels, outputs, classes)

0.25

In [19]:
labels = np.array([[0,0,1],[0,1,0]])
outputs = np.array([[0,0,1],[1,0,0]])
classes = ['Present', 'Unknown', 'Absent']

In [10]:

# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy
